In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Fri Nov 11 16:10:35 2022

@author: rubby
"""

import twitter
import json
import time
import pandas as pd
import networkx as nx
import pickle
import random
import config2 as config
import os.path
import time
from datetime import datetime

twitter_api = twitter.Api(consumer_key=config.twitter_consumer_key,
                              consumer_secret=config.twitter_consumer_secret, 
                              access_token_key=config.twitter_access_token, 
                              access_token_secret=config.twitter_access_secret,
                              sleep_on_rate_limit=True)

most_followed = pd.read_csv('crawling/top100.csv')


def collecting_mention(artist_name, artist_account):
    filepath_n = ('data/network/{}.net'.format(artist_name)).replace('*', ' ').replace('!', '')
    filepath_f = ('data/followers_filtered/{}.p'.format(artist_name)).replace('*', ' ').replace('!', '')
    dir_t = ('data/tweets/{}'.format(artist_name)).replace('*', ' ').replace('!', '')
    avg_tweet_num = 0
    if not os.path.exists(dir_t):
        os.makedirs(dir_t)
    
    if not os.path.isfile(filepath_n):
        with open(filepath_f, 'rb') as f:
            followers = pickle.load(f)
        
        G = nx.Graph()
        
        for follower in followers:
            filepath_t = ('data/tweets/{}/{}.p'.format(artist_name, follower.id)).replace('*', ' ').replace('!', '')
            all_statuses = []
            print('artist: {}, follower: {}/{}'.format(artist_name, followers.index(follower)+1, len(followers)))
            
            last_date = datetime.now()
            last_id = None
            
            try:
                # for test, initialization
                statuses = twitter_api.GetUserTimeline(screen_name=artist_account)
            except Exception as e:
                print(e)
            
            while last_date > datetime(2022,5,1) or statuses:
                try:
                    statuses = twitter_api.GetUserTimeline(user_id=follower.id, max_id=last_id, count=200)
                except Exception as e:
                    print('In GetUserTimeline: {}'.format(e))
                    statuses = []
                    time.sleep(5)
                if statuses:
                    last_status = statuses[-1]
                    last_date = datetime.strptime(last_status.created_at,'%a %b %d %H:%M:%S +0000 %Y')
                    last_id = last_status.id
                    
                    for status in statuses:
                        s_created_date = datetime.strptime(status.created_at,'%a %b %d %H:%M:%S +0000 %Y')
                        
                        if s_created_date < datetime(2022,11,1) and s_created_date >= datetime(2022,5,1):
                            all_statuses.append([status.id, s_created_date, status.text])
                            if status.user_mentions:
                                for mentioned_user in status.user_mentions:
                                    if mentioned_user.id != follower.id:
                                        try:
                                            following = twitter_api.ShowFriendship(source_user_id=mentioned_user.id, 
                                                                                   target_screen_name=artist_account)

                                            if following.get('relationship').get('source').get('following'):
                                                G.add_edge(follower.id, mentioned_user.id)
                                                nx.write_pajek(G, filepath_n)
                                                #print('add edge of {}. date: {}'.format(artist_name, status.created_at))
                                        except Exception as e:
                                            print('In ShowFriendship: {}'.format(e))
                if len(statuses) < 200:
                    break
            with open(filepath_t, 'wb') as f:
                pickle.dump(all_statuses, f)
            
            avg_tweet_num += len(all_statuses) / len(followers)
        
        print('average tweets of {}: {}'.format(artist_name, avg_tweet_num))
    else:
        print('{} network exists'.format(artist_name))


for i in most_followed.index:
    name = most_followed['name'][i]
    acc = most_followed['account'][i]
    collecting_mention(name, acc)

BTS network exists
artist: EXO, follower: 1/100
In GetUserTimeline: Not authorized.
artist: EXO, follower: 2/100
artist: EXO, follower: 3/100
artist: EXO, follower: 4/100
artist: EXO, follower: 5/100
artist: EXO, follower: 6/100
artist: EXO, follower: 7/100
In GetUserTimeline: Not authorized.
artist: EXO, follower: 8/100
In GetUserTimeline: Not authorized.
artist: EXO, follower: 9/100
artist: EXO, follower: 10/100
artist: EXO, follower: 11/100
artist: EXO, follower: 12/100
In GetUserTimeline: Not authorized.
artist: EXO, follower: 13/100
In GetUserTimeline: Not authorized.
artist: EXO, follower: 14/100
artist: EXO, follower: 15/100
artist: EXO, follower: 16/100
artist: EXO, follower: 17/100
artist: EXO, follower: 18/100
artist: EXO, follower: 19/100
artist: EXO, follower: 20/100
artist: EXO, follower: 21/100
In ShowFriendship: [{'code': 163, 'message': 'Could not determine source user.'}]
artist: EXO, follower: 22/100
artist: EXO, follower: 23/100
artist: EXO, follower: 24/100
In GetUs